In [ ]:
function correlation_matrix(Lx, n_occupied)
    
    # create Hamiltonian matrix for hopping fermions
    t = 1.
    mu = 0.
    Hmatrix = SymTridiagonal(mu .* ones(Lx), t .* ones(Lx-1))

    # Make the correlation matrix (\Lambda) from its diagonal form (\Gamma)
    # \Gamma has n_occupied 1s--corresponding to the lowest eigenvalues--and the rest of the diagonal 0s
    # we have $\Lambda = U^{*} \Gamma U^{T}$
    vecs = eigfact(Hmatrix, 1:n_occupied)[:vectors]
    return conj(vecs) * transpose(vecs)
    
end

In [75]:
Lx = 20
n_occupied = 10
corr_matrix = correlation_matrix(Lx, n_occupied)

## TODO: make the size dynamic for better efficiency here (how does this translate to GDMRG?!)
## TODO: better slicing method (probably arrayviews or sub)
for site=1:Lx
    # find the best block size
    if site < Lx-5
        bsize = 6
    else
        bsize = Lx - site + 1
    end
    brange = site:bsize+site-1
    corr_block = corr_matrix[brange, brange]

    # find the closest eigenvalue to occupied/unoccupied and the corresponding eigenvector
    vals = eigvals(corr_block)
    v_index = indmax(abs.(vals - 0.5))
    v = eigvecs(corr_block)[:,v_index]

    # find the set of bsize-1 unitary gate that diagonalize the correlation block
    ## TODO: how to organize and store the gates?
    fullgate = eye(bsize, bsize)
    for pivot=bsize-1:-1:1
        theta = atan(v[pivot + 1]/ v[pivot])
        Ugate = eye(bsize, bsize)
        Ugate[pivot:pivot+1, pivot:pivot+1] = [cos(theta) -sin(theta); sin(theta) cos(theta)]
        fullgate = fullgate * Ugate
        v = transpose(v.' * Ugate)
    end
    
    # rotate the corresponding correlation block
    corr_matrix[brange, brange] = fullgate' * corr_block * fullgate
end
corr_matrix
## how to apply the gates the most efficient way?
## how to implement Gutzwiller projection in this approach?

20×20 Array{Float64,2}:
  1.0          -1.5168e-16    7.58942e-17  …  -1.49186e-16   0.047888   
 -1.2837e-16   -0.034642      3.54805e-17     -0.00110697   -1.78677e-16
 -1.64799e-17   4.51028e-17   1.0989           4.68375e-17  -0.048995   
 -1.48319e-16  -2.1684e-17   -7.33788e-16      0.00234812    1.17961e-16
 -6.07153e-17  -7.97973e-17   1.52656e-16     -1.52656e-16   0.0513431  
 -1.38778e-17   4.67291e-17  -9.02056e-17  …  -0.00390027   -2.42861e-16
 -5.55112e-17  -2.77556e-17  -5.55112e-17      1.07553e-16  -0.0552433  
  0.0858066     2.60209e-17  -1.94289e-16      0.00605342    9.71445e-17
  1.80411e-16   0.015139     -4.16334e-17     -6.07153e-17   0.0612968  
 -0.0706675     1.52656e-16   0.0764358       -0.00937075    1.52656e-16
 -2.63678e-16  -0.00937075    1.66533e-16  …   6.59195e-17  -0.0706675  
  0.0612968    -1.94289e-16  -0.0646141        0.015139     -1.38778e-17
  6.93889e-18   0.00605342   -7.63278e-17     -2.28983e-16   0.0858066  
 -0.0552433     3.64292e-17

In [ ]:
# generate MPS from the GMPS

# start with a product state of correct occupation number
# apply the nearest neighbor gates (opposite order) and form new MPS svd